In [3]:
!pip install bs4

In [4]:
!pip install selenium

In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
def get_player_rank(server,name,tagline):
    opgg_url = f"https://{server}.op.gg/summoner/userName={name}-{tagline}"

    try:
        response = requests.get(opgg_url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        rank_element = soup.find(class_='tier')

        if rank_element:
            rank = rank_element.text.strip()
            return rank
        else:
            print("Rank information not found for this player.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [10]:
name = "biadee"
tagline = "eune"
server = "eune"
player_rank = get_player_rank(server,name,tagline)
if player_rank:
    print(f"{name}#{tagline} is ranked as: {player_rank}")

biadee#eune is ranked as: emerald 1


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF66EEEAD02+56930]
	(No symbol) [0x00007FF66EE5F602]
	(No symbol) [0x00007FF66ED142E5]
	(No symbol) [0x00007FF66ED598ED]
	(No symbol) [0x00007FF66ED59A2C]
	(No symbol) [0x00007FF66ED9A967]
	(No symbol) [0x00007FF66ED7BCDF]
	(No symbol) [0x00007FF66ED981E2]
	(No symbol) [0x00007FF66ED7BA43]
	(No symbol) [0x00007FF66ED4D438]
	(No symbol) [0x00007FF66ED4E4D1]
	GetHandleVerifier [0x00007FF66F266F8D+3711213]
	GetHandleVerifier [0x00007FF66F2C04CD+4077101]
	GetHandleVerifier [0x00007FF66F2B865F+4044735]
	GetHandleVerifier [0x00007FF66EF89736+706710]
	(No symbol) [0x00007FF66EE6B8DF]
	(No symbol) [0x00007FF66EE66AC4]
	(No symbol) [0x00007FF66EE66C1C]
	(No symbol) [0x00007FF66EE568D4]
	BaseThreadInitThunk [0x00007FFD6A28257D+29]
	RtlUserThreadStart [0x00007FFD6C52AA58+40]


2024-03-19 10:52:04 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): lolalytics.com:443
2024-03-19 10:52:04 [urllib3.connectionpool] DEBUG: https://lolalytics.com:443 "GET /lol/tierlist/ HTTP/1.1" 200 None


None
Broken champions information not found.
